In [1]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2023-07-06 11:16:16.417693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

2.12.0
1.4.0


In [20]:
model_tree_type=2 # 1= xgboost  2-random forest
option=2

if model_tree_type==1:
    _model='model_xgb_tf'
else:
     _model='model_rf_tf'

model_gs_path=f"gs://demo-tuned-tf-incident-pongthorn/{_model}"
print(model_gs_path)

projectId="pongthorn"
dataset_id="SMartML"

map_severity_to_class={0:4,1: 3, 2: 2, 3: 1}

if option==1:
    unusedCols_unseen=['id','severity_name','open_to_close_hour']
else:
    unusedCols_unseen=['id','severity_name','range_open_to_close_hour']
    

gs://demo-tuned-tf-incident-pongthorn/model_rf_tf


# Load Model

In [21]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-07-06 11:20:11.0687 UTC kernel.cc:1243] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_rf_tf/assets/ with prefix e3168b6d1bdc44ce
[INFO 23-07-06 11:20:11.8151 UTC decision_forest.cc:660] Model loaded with 300 root(s), 140518 node(s), and 6 input feature(s).
[INFO 23-07-06 11:20:11.8160 UTC abstract_model.cc:1311] Engine "RandomForestGeneric" built
[INFO 23-07-06 11:20:11.8165 UTC kernel.cc:1075] Use fast generic engine


Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# Make Prediction

In [22]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
df=query_result.to_dataframe()

unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   severity_id         17 non-null     Int64  
 1   sla                 17 non-null     object 
 2   product_type        17 non-null     object 
 3   brand               17 non-null     object 
 4   service_type        17 non-null     object 
 5   incident_type       17 non-null     object 
 6   open_to_close_hour  17 non-null     float64
dtypes: Int64(1), float64(1), object(5)
memory usage: 1.1+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour
7,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,34.250000
8,3,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Controller/Node Failure,34.250000
9,3,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Hard Disk Drive Failure,24.500000
10,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,97.500000
11,4,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,89.433333
12,3,24x7 4Hrs Response Time,Security,Trend Micro,Incident,Upgrade Software,59.000000
13,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,2.300000
14,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,1.233333
15,3,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,6.316667
16,2,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,23.483333


In [23]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
unseen_ds

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>

In [24]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    pred_seveirty_id=map_severity_to_class[_class]
    predServerityIDList.append(pred_seveirty_id)
    print(f"{predResult} : {_class} as severity#{pred_seveirty_id}")
    
dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_severity_id"])   
df=pd.concat([dfPred,df],axis=1)

1/1 [==============================] - 0s 72ms/step
[0.97666585 0.01666667 0.00333333 0.00333333] : 0 as severity#4
[0.         0.01333333 0.98666584 0.        ] : 2 as severity#2
[0.         0.08999998 0.88666594 0.02333334] : 2 as severity#2
[0.         0.00666667 0.9933325  0.        ] : 2 as severity#2
[0.01333333 0.12666662 0.816666   0.04333334] : 2 as severity#2
[0.         0.04666667 0.95333254 0.        ] : 2 as severity#2
[0.00333333 0.3533331  0.63333285 0.01      ] : 2 as severity#2
[0.05666667 0.29666647 0.40666637 0.23999985] : 2 as severity#2
[0.03666667 0.21999986 0.41333303 0.32999977] : 2 as severity#2
[0.         0.67666614 0.31666645 0.00666667] : 1 as severity#3
[0.09999997 0.12666662 0.7733327  0.        ] : 2 as severity#2
[0.         0.52333295 0.4766663  0.        ] : 1 as severity#3
[0.20333321 0.5366663  0.25333318 0.00666667] : 1 as severity#3
[0.         0.2566665  0.74333274 0.        ] : 2 as severity#2
[0.         0.6833328  0.31666645 0.        ] : 1 as

2023-07-06 11:20:20.005890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [17]
	 [[{{node Placeholder/_5}}]]


In [25]:
from sklearn.metrics import confusion_matrix,classification_report
className=list(set().union(list(df['pred_severity_id'].unique()),list(df['severity_id'].unique())))
className
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'pred-{x}' for x in className]
y_true=list(df['severity_id'])
y_pred=list(df['pred_severity_id'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
cm_df

,pred-2,pred-3,pred-4
actual-2,4,1,0
actual-3,5,4,0
actual-4,1,1,1


In [26]:
print(classification_report(y_true, y_pred, labels=className))

              precision    recall  f1-score   support

           2       0.40      0.80      0.53         5
           3       0.67      0.44      0.53         9
           4       1.00      0.33      0.50         3

    accuracy                           0.53        17
   macro avg       0.69      0.53      0.52        17
weighted avg       0.65      0.53      0.53        17

